# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [4]:
df = pd.DataFrame(pd.read_excel('334data.xlsx')) 
df.head()

,Part ID,Planning Leadtime,Order Policy,Order Point,Safety Stock Qty,Fixed Order Qty,Days Of Supply,Minimum Order Qty,Maximum Order Qty,Multiple Order Qty,...,Yr2011,Yr2012,Yr2013,Yr2014,Turns30,Demand 30day,Turns60,Demand 60day,Turns90,Demand 90day
0,3.290050e+09,21,P,NaN,2400.0,NaN,28.0,2400.0,NaN,2400.0,...,240104.0,118284.0,119960.0,204486.00,15.940000,19128.00,7.886667,18928.0,0.000000,0.00
1,3.290400e+09,90,P,NaN,5400.0,NaN,28.0,NaN,NaN,540.0,...,108576.0,81707.0,109442.0,27938.00,13.241128,7276.00,6.511374,7156.0,0.000000,0.00
2,3.271255e+09,42,P,NaN,NaN,NaN,30.0,12960.0,NaN,432.0,...,79033.0,121980.0,50539.0,62167.22,25.981835,13313.85,0.421522,432.0,2.701134,4152.42
3,3.290051e+09,21,P,NaN,4800.0,NaN,30.0,4800.0,NaN,4800.0,...,NaN,NaN,NaN,NaN,19.990000,7996.00,4.370000,3496.0,6.913333,8296.00
4,3.265400e+09,21,N,2000.0,NaN,NaN,NaN,8000.0,NaN,1000.0,...,51000.0,57000.0,36000.0,36000.00,99999.000000,1.00,99999.000000,1.0,99999.000000,0.00


In [18]:
len(df)

88051

In [19]:
df2 = df[(df['Pur'] == 'Y') & (df['Def Orig Country'] == 'CA')]
df2.head()

,Part ID,Planning Leadtime,Order Policy,Order Point,Safety Stock Qty,Fixed Order Qty,Days Of Supply,Minimum Order Qty,Maximum Order Qty,Multiple Order Qty,...,Yr2011,Yr2012,Yr2013,Yr2014,Turns30,Demand 30day,Turns60,Demand 60day,Turns90,Demand 90day
2,3.271255e+09,42,P,NaN,NaN,NaN,30.0,12960.0,NaN,432.0,...,79033.0,121980.0,50539.0,62167.22,25.981835,13313.85,0.421522,432.00,2.701134,4152.42
6,3.270115e+09,21,P,NaN,NaN,NaN,30.0,3810.0,NaN,762.0,...,8518.0,3380.0,NaN,-6674.00,3.685354,1170.10,1.332598,846.20,0.010499,10.00
7,3.270115e+09,21,P,NaN,NaN,NaN,30.0,3810.0,NaN,762.0,...,8434.0,3533.0,NaN,-7436.00,3.562205,1131.00,1.289764,819.00,0.000000,0.00
8,3.270115e+09,21,P,NaN,NaN,NaN,30.0,3810.0,NaN,762.0,...,8123.0,3685.0,NaN,-7536.00,3.562205,1131.00,1.289764,819.00,0.000000,0.00
9,3.270115e+09,21,P,NaN,NaN,NaN,30.0,3810.0,NaN,762.0,...,8267.0,3685.0,NaN,-7381.00,3.562205,1131.00,1.292535,820.76,0.000000,0.00


In [20]:
len(df2)

2412

In [22]:
# Calculating the percentage of missing values in each column
missing_percentage = (df2.isnull().sum() / len(df2)) * 100

# Filtering out the percentages for the relevant columns identified for forecasting
relevant_columns = [
    'Yr2010', 'Yr2011', 'Yr2012', 'Yr2013', 'Yr2014',
    'Qty On Hand', 'Qty Available Mrp', 'Planning Leadtime', 'Purc Lt',
    'Minimum Order Qty', 'Maximum Order Qty', 'Multiple Order Qty',
    'Safety Stock Qty', 'Fixed Order Qty', 'Annual Usage Qty', 'Prev Yr Usage',
    'Roll Ann Usage', 'Turns30', 'Demand 30day', 'Turns60', 'Demand 60day',
    'Turns90', 'Demand 90day', 'Mean', 'Stddev', 'Coeff'
]

missing_percentage_relevant = missing_percentage[relevant_columns]
missing_percentage_relevant

Yr2010                 62.852405
Yr2011                 62.603648
Yr2012                 62.106136
Yr2013                 66.293532
Yr2014                 70.190713
Qty On Hand             0.000000
Qty Available Mrp       0.000000
Planning Leadtime       0.000000
Purc Lt                 0.331675
Minimum Order Qty      56.965174
Maximum Order Qty      97.636816
Multiple Order Qty     55.597015
Safety Stock Qty       92.620232
Fixed Order Qty        87.935323
Annual Usage Qty        0.000000
Prev Yr Usage           0.124378
Roll Ann Usage          0.000000
Turns30                 0.000000
Demand 30day            0.000000
Turns60                 0.000000
Demand 60day            0.000000
Turns90                 0.000000
Demand 90day            0.000000
Mean                  100.000000
Stddev                100.000000
Coeff                 100.000000
dtype: float64

In [23]:
# Removing records with missing values in the specified columns
# Also, selecting only the columns from Yr2010 to Yr2014 as the user requested no other years.
columns_to_keep = ['Part ID', 'Planning Leadtime', 'Order Policy', 'Order Point', 'Safety Stock Qty',
                   'Fixed Order Qty', 'Days Of Supply', 'Minimum Order Qty', 'Maximum Order Qty',
                   'Multiple Order Qty', 'Fab', 'Pur', 'Stocked', 'Detail Only', 'Purc Lt', 
                   'Strategic Flag', 'Abc Code', 'Inventory Locked', 'Inventory$', 'Unit Material Cost', 
                   'Annual Usage Qty', 'Prev Yr Usage', 'Roll Ann Usage', 'Qty On Hand', 
                   'Qty Available Mrp', 'TotAlloc', 'AvgBal', 'Mean', 'Stddev', 'Coeff', 
                   'Coeff_Code', 'Stock Um', 'Def Orig Country', 'Primary Whs Id', 'Primary Loc Id',
                   'Turns30', 'Demand 30day', 'Turns60', 'Demand 60day', 'Turns90', 'Demand 90day',
                   'Yr2010', 'Yr2011', 'Yr2012', 'Yr2013', 'Yr2014']

df3 = df2[columns_to_keep].dropna(subset=['Yr2010', 'Yr2011', 'Yr2012', 'Yr2013', 'Yr2014'])

df3.head()


,Part ID,Planning Leadtime,Order Policy,Order Point,Safety Stock Qty,Fixed Order Qty,Days Of Supply,Minimum Order Qty,Maximum Order Qty,Multiple Order Qty,...,Demand 30day,Turns60,Demand 60day,Turns90,Demand 90day,Yr2010,Yr2011,Yr2012,Yr2013,Yr2014
2,3.271255e+09,42,P,NaN,NaN,NaN,30.0,12960.0,NaN,432.0,...,13313.85,0.421522,432.00,2.701134,4152.42,146609.0,79033.0,121980.0,50539.0,62167.22
25,3.276750e+09,21,P,NaN,NaN,NaN,60.0,100.0,NaN,100.0,...,494.00,1.686000,281.00,0.524000,131.00,254.0,1748.0,3800.0,4200.0,4500.00
26,3.275715e+09,21,P,NaN,NaN,NaN,60.0,1000.0,NaN,1000.0,...,1076.20,2.462160,410.36,1.061440,265.36,2006.0,2994.0,3000.0,3400.0,2700.00
34,3.276890e+09,14,P,NaN,NaN,NaN,60.0,25.0,NaN,25.0,...,1599.00,1.593000,531.00,1.624000,812.00,54.0,1845.0,300.0,1350.0,3000.00
76,3.270125e+09,42,P,NaN,NaN,NaN,35.0,500.0,NaN,NaN,...,640.00,6.868825,672.00,0.408859,60.00,2006.0,1810.0,3112.0,840.0,2194.00


In [24]:
len(df3)

434